In [1]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from glob import glob
import pickle
import nltk
from torch.nn.utils.rnn import pack_padded_sequence
import timm
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils.rnn import pad_packed_sequence
from efficientnet_pytorch_3d import EfficientNet3D

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('pun`kt_tab')

device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to /home/gil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/gil/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Error loading pun`kt_tab: Package 'pun`kt_tab' not found
[nltk_data]     in index


In [31]:
params={'image_size':512,
        'lr':1e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':5,
        'epochs':300,
        'image_count':50,
        'data_path':'../data/NIA7/',
        'train_csv':'fold_5_train.csv',
        'val_csv':'fold_5_val.csv',
        'vocab_path':'../data/vocab.pkl',
        'embed_size':300,
        'hidden_size':256,
        'num_layers':1,}

In [3]:
class CustomDataset(Dataset):
    def __init__(self, data_path, image_count, image_size, csv, class_dataset, vocab, transform=None):
        self.root = data_path
        self.image_count = image_count
        self.df = pd.read_csv(data_path + csv)
        self.class_dataset = class_dataset
        self.vocab = vocab
        self.transform = transform
        self.image_size = image_size

        # 'age', 'height', 'weight' 컬럼을 MinMaxScaler로 스케일링
        self.df['sex'] = self.df['sex'].map({'M': 0, 'F': 1})
        self.scaler = MinMaxScaler()
        self.df[['age', 'sex', 'height', 'weight']] = self.scaler.fit_transform(self.df[['age', 'sex', 'height', 'weight']])
        
        # 모든 데이터를 메모리에 캐싱
        self.cached_data = []
        for index, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Caching data"):
            patient_id = row['ID']
            
            # CT 이미지(3D 데이터) 로드 및 캐싱
            ct_image_paths = glob(self.root + f'{patient_id}/02. CT/*.png')
            ct_images = torch.zeros(self.image_count, 1, self.image_size, self.image_size)
            ct_image_index = torch.randint(low=0, high=len(ct_image_paths), size=(self.image_count,))
            for count, idx in enumerate(ct_image_index):
                ct_image = Image.open(ct_image_paths[idx]).convert('L')
                if self.transform is not None:
                    ct_image = self.transform(ct_image)
                ct_images[count] = ct_image

            ct_images = ct_images.permute(1, 0, 2, 3)  # 형태: (1, image_count, image_size, image_size)

            # CR 이미지(2D 데이터) 로드 및 캐싱
            cr_image_path = glob(self.root + f'{patient_id}/01. CR/*.png')[0]
            cr_image = Image.open(cr_image_path).convert('L')
            if self.transform is not None:
                cr_image = self.transform(cr_image)

            # 캡션을 토큰 ID로 변환하여 캐싱
            caption = row['text']
            tokens = nltk.tokenize.word_tokenize(str(caption).lower())
            caption_tokens = [vocab('<start>')] + [vocab(token) for token in tokens] + [vocab('<end>')]
            target = torch.Tensor(caption_tokens)

            # 'age', 'height', 'weight' 값 캐싱
            sex = torch.tensor(row['sex'], dtype=torch.float32)
            age = torch.tensor(row['age'], dtype=torch.float32)
            height = torch.tensor(row['height'], dtype=torch.float32)
            weight = torch.tensor(row['weight'], dtype=torch.float32)

            # 캐싱된 데이터 저장
            self.cached_data.append((ct_images, cr_image, target, sex, age, height, weight))

    def __getitem__(self, index):
        # 캐싱된 데이터에서 인덱스로 직접 접근
        return self.cached_data[index]

    def __len__(self):
        return len(self.df)

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def collate_fn(data):
    # Sort data by caption length (descending order).
    data.sort(key=lambda x: len(x[2]), reverse=True)
    ct_images, cr_images, captions, sexs, ages, heights, weights = zip(*data)

    # Merge images
    ct_images = torch.stack(ct_images, 0)
    cr_images = torch.stack(cr_images, 0)

    # Merge captions
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]

    # Stack ages and heights
    sexs = torch.stack(sexs, 0)
    ages = torch.stack(ages, 0)
    heights = torch.stack(heights, 0)
    weights = torch.stack(weights, 0)

    return ct_images, cr_images, targets, lengths, sexs, ages, heights, weights

def idx2word(vocab, indices):
    sentence = []
    for i in range(params['batch_size']):
        indices[i].cpu().numpy()
    
    for index in indices:
        word = vocab.idx2word[index]
        sentence.append(word)
    return sentence

In [23]:
# ======= Encoder - 2D Feature Extractor (CNN) =======
class FeatureExtractor2D(nn.Module):
    """2D Feature extractor block for X-ray images (simple CNN version)"""
    def __init__(self, dropout_rate=0.3):
        super(FeatureExtractor2D, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2d_2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv2d_3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # 2x downsampling
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)  # Dropout layer 추가

    def forward(self, inputs):
        x = self.conv2d_1(inputs)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)  # Apply dropout

        x = self.conv2d_2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)  # Apply dropout

        x = self.conv2d_3(x)
        x = F.relu(x)
        x = self.pool(x)
        features = self.flatten(x)
        features = self.dropout(features)  # Apply dropout at the end
        return features
    

# ======= Encoder - 3D Feature Extractor =======
class FeatureExtractor3D(nn.Module):
    """3D Feature extractor block for CT images"""
    def __init__(self, dropout_rate=0.3):
        super(FeatureExtractor3D, self).__init__()
        self.conv3d_1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.conv3d_2 = nn.Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        self.conv3d_3 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        self.pool = nn.MaxPool3d(2)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout_rate)  # Dropout layer 추가

    def forward(self, inputs):
        x = self.conv3d_1(inputs)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)  # Apply dropout

        x = self.conv3d_2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)  # Apply dropout

        x = self.conv3d_3(x)
        x = F.relu(x)
        x = self.pool(x)
        features = self.flatten(x)  # Flatten the output to a 2D tensor
        features = self.dropout(features)  # Apply dropout at the end
        return features


In [24]:
class AttentionMILModel(nn.Module):
    def __init__(self, image_feature_dim_2d, image_feature_dim_3d, 
                 feature_extractor_2d: nn.Module, feature_extractor_3d: nn.Module,
                 age_height_weight_input_size=4, dropout_rate=0.3):
        super(AttentionMILModel, self).__init__()
        self.feature_extractor_2d = feature_extractor_2d
        self.feature_extractor_3d = feature_extractor_3d

        self.age_height_weight_mlp = nn.Sequential(
            nn.Linear(age_height_weight_input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        self.combined_feature_dim = image_feature_dim_2d + image_feature_dim_3d + 32
        self.dropout = nn.Dropout(dropout_rate)

        # Attention + projection for T5
        self.attention_2d = nn.Sequential(
            nn.Linear(self.combined_feature_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        self.feature_proj = nn.Linear(self.combined_feature_dim, 512)  # ✅ T5 input projection

    def forward(self, inputs_2d, inputs_3d, age_height_weight):
        features_2d = self.feature_extractor_2d(inputs_2d)
        features_3d = self.feature_extractor_3d(inputs_3d)
        features_meta = self.age_height_weight_mlp(age_height_weight)

        features = torch.cat([features_2d, features_3d, features_meta], dim=1)

        attention_weights = self.attention_2d(features)
        attention_weights = F.softmax(attention_weights, dim=1)
        attention_weights = self.dropout(attention_weights)

        attended = features * attention_weights
        attended = self.dropout(attended)

        projected = self.feature_proj(attended)  # [B, 512]
        return projected  # ❗ T5 decoder에 넘기기 전에는 .unsqueeze(1) 필요


In [25]:
from transformers import T5Model, T5Config
import torch.nn as nn
import torch.nn.functional as F
import torch

class SwiGLUFFN(nn.Module):
    def __init__(self, hidden_size, intermediate_size, dropout_rate=0.1):
        super(SwiGLUFFN, self).__init__()
        self.gate_proj = nn.Linear(hidden_size, intermediate_size)
        self.up_proj = nn.Linear(hidden_size, intermediate_size)
        self.down_proj = nn.Linear(intermediate_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        gate = F.silu(self.gate_proj(x))
        up = self.up_proj(x)
        x = gate * up
        x = self.down_proj(x)
        x = self.dropout(x)
        return x

class ReducedDecoderT5SwiGLU(nn.Module):
    def __init__(self, vocab_size, max_seq_length=50, dropout_rate=0.7):
        super(ReducedDecoderT5SwiGLU, self).__init__()
        self.max_seq_length = max_seq_length

        config = T5Config(
            vocab_size=vocab_size,
            d_model=512,
            d_ff=2048,
            num_layers=6,
            num_heads=8,
            dropout_rate=dropout_rate,
            is_encoder_decoder=True,
            output_attentions=True,
            return_dict=True
        )

        self.transformer = T5Model(config)
        self.lm_head = nn.Linear(config.d_model, vocab_size, bias=False)

        # Replace FFN with SwiGLUFFN
        for block in self.transformer.decoder.block:
            hidden_size = config.d_model
            intermediate_size = config.d_ff
            block.layer[1].DenseReluDense = SwiGLUFFN(hidden_size, intermediate_size, dropout_rate)

    def forward(self, encoder_hidden_states, decoder_input_ids, decoder_attention_mask=None):
        decoder_outputs = self.transformer(
            encoder_outputs=(encoder_hidden_states,),  # <-- 핵심
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask
        )
        sequence_output = decoder_outputs.last_hidden_state
        logits = self.lm_head(sequence_output)
        return logits


In [26]:
with open(params['vocab_path'], 'rb') as f:
        vocab = pickle.load(f)

transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))  # Adjust for grayscale images with 1 channel
])

In [27]:
Feature_Extractor2D = FeatureExtractor2D()
Feature_Extractor3D = FeatureExtractor3D()

# Initialize encoder and new decoder
encoder = AttentionMILModel(65536, 196608, Feature_Extractor2D, Feature_Extractor3D).to(device)
decoder = ReducedDecoderT5SwiGLU(len(vocab)).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
model_param = list(decoder.parameters()) + list(encoder.parameters())
optimizer = torch.optim.Adam(model_param, lr=params['lr'], betas=(params['beta1'], params['beta2']), weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

In [28]:
# EfficientNet B3의 FeatureExtractor2D 출력 크기 확인
sample_input = torch.randn(1, 1, params['image_size'], params['image_size']).to(device)
features_2d = Feature_Extractor2D(sample_input)
print("FeatureExtractor2D feature size:", features_2d.shape)

FeatureExtractor2D feature size: torch.Size([1, 65536])


In [29]:
# FeatureExtractor3D의 출력 크기 확인
sample_ct_input = torch.randn(1, 1, params['image_count'], params['image_size'], params['image_size']).to(device)
features_3d = Feature_Extractor3D(sample_ct_input)
print("FeatureExtractor3D output size:", features_3d.shape)

FeatureExtractor3D output size: torch.Size([1, 196608])


# caching data

In [13]:
train_dataset=CustomDataset(params['data_path'],params['image_count'],params['image_size'],params['train_csv'],'train',vocab,transform=transform)
test_dataset=CustomDataset(params['data_path'],params['image_count'],params['image_size'],params['val_csv'],'val',vocab,transform=transform)
train_dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)
val_dataloader=DataLoader(test_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)

Caching data: 100%|██████████| 163/163 [05:04<00:00,  1.87s/it]


In [ ]:
best_val_loss = float("inf")

for epoch in range(params['epochs']):
    # Training loop
    train = tqdm(train_dataloader)
    count = 0
    train_loss = 0.0
    decoder.train()  # Ensure the decoder is in training mode
    encoder.train()
    
    for ct_images, cr_images, captions, lengths, sexs, ages, heights, weights in train:
        ct_images, cr_images, captions = ct_images.to(device), cr_images.to(device), captions.to(device)
        sexs, ages, heights, weights = sexs.to(device), ages.to(device), heights.to(device), weights.to(device)
        count += 1

        # Forward pass for encoder
        features = encoder(cr_images, ct_images, torch.stack([sexs, ages, heights, weights], dim=1))
        features = features.unsqueeze(1)  # [B, D] → [B, 1, D]
        
        # Prepare inputs for T5 decoder
        labels = captions[:, 1:]          # Remove the first token for labels

        # Forward pass through decoder with T5 format
        outputs = decoder(encoder_hidden_states=features, decoder_input_ids=labels)
        loss = criterion(outputs.view(-1, outputs.size(-1)), labels.contiguous().view(-1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

        train.set_description(f"epoch: {epoch+1}/{params['epochs']} Step: {count+1} loss : {train_loss/count:.4f} ")

    # Validation loop
    with torch.no_grad():
        val_count = 0
        val_loss = 0.0
        val = tqdm(val_dataloader)
        decoder.eval()  # Ensure the decoder is in evaluation mode
        encoder.eval()
        
        for ct_images, cr_images, captions, lengths, sexs, ages, heights, weights in val:
            val_count += 1
            ct_images, cr_images, captions = ct_images.to(device), cr_images.to(device), captions.to(device)
            sexs, ages, heights, weights = sexs.to(device), ages.to(device), heights.to(device), weights.to(device)

            # Forward pass for validation
            features = encoder(cr_images, ct_images, torch.stack([sexs, ages, heights, weights], dim=1))
            features = features.unsqueeze(1)  # [B, D] → [B, 1, D]
            
            # Validation inputs
            labels = captions[:, 1:]

            # Validation forward pass through decoder
            outputs = decoder(encoder_hidden_states=features, decoder_input_ids=labels)
            loss = criterion(outputs.view(-1, outputs.size(-1)), labels.contiguous().view(-1))
            val_loss += loss.item()
            val.set_description(f"epoch: {epoch+1}/{params['epochs']} Step: {val_count+1} loss : {val_loss/val_count:.4f} ")
        
        # Average validation loss
        avg_val_loss = val_loss / len(val_dataloader)
        print(f"Epoch [{epoch+1}/{params['epochs']}], Validation Loss: {avg_val_loss:.4f}")
        print("Current learning rate:", optimizer.param_groups[0]['lr'])

        # Call scheduler with validation loss
        scheduler.step(avg_val_loss)

        # Save the model if it has the best validation loss so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(encoder.state_dict(), '../model/test_att_encoder.pth')
            torch.save(decoder.state_dict(), '../model/test_att_decoder.pth')
            print("Best model saved with Validation Loss:", best_val_loss)


  0%|          | 0/130 [00:00<?, ?it/s]

epoch: 1/300 Step: 34 loss : 0.1400 : 100%|██████████| 33/33 [00:06<00:00,  4.86it/s]


Epoch [1/300], Validation Loss: 0.1400
Current learning rate: 0.0001
Best model saved with Validation Loss: 4.620517782866955


epoch: 2/300 Step: 34 loss : 0.1332 : 100%|██████████| 33/33 [00:06<00:00,  4.91it/s]


Epoch [2/300], Validation Loss: 0.1332
Current learning rate: 0.0001
Best model saved with Validation Loss: 4.394742090255022


...

epoch: 60/300 Step: 34 loss : 0.0834 : 100%|██████████| 33/33 [00:06<00:00,  4.76it/s]


Epoch [60/300], Validation Loss: 0.0834
Current learning rate: 1.220703125e-08


epoch: 61/300 Step: 34 loss : 0.0829 : 100%|██████████| 33/33 [00:06<00:00,  5.07it/s]


Epoch [61/300], Validation Loss: 0.0829
Current learning rate: 1.220703125e-08


epoch: 62/300 Step: 43 loss : 0.2297 :  32%|███▏      | 42/130 [00:32<01:08,  1.28it/s]